In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext line_profiler

In [3]:
import itertools
import torch

from tsvar.simulate import GrangerBuscaSimulator
from tsvar.wold_model import VariationalWoldModel

---

## Generate small toy example dataset

Define model parameters

In [4]:
dim = 3

baseline = np.array([0.1, 0.1, 0.2])

# adjacency[i,j] = magnitude of influence from i to j
adjacency = np.array([
    [0.3, 0.8, 0.1],
    [0.2, 0.3, 0.1],
    [0.2, 0.1, 0.0]
])

beta = 1.0

end_time = 1e5

for i, j in itertools.product(range(dim), repeat=2):
    print(f"w({j:d} -> {i:d}) = {adjacency[j,i]:.2f}")

w(0 -> 0) = 0.30
w(1 -> 0) = 0.20
w(2 -> 0) = 0.20
w(0 -> 1) = 0.80
w(1 -> 1) = 0.30
w(2 -> 1) = 0.10
w(0 -> 2) = 0.10
w(1 -> 2) = 0.10
w(2 -> 2) = 0.00


Simulate a dataset

In [5]:
wold_sim = GrangerBuscaSimulator(mu_rates=baseline, Alpha_ba=adjacency, Beta_b=beta * np.ones((dim,)))
events = wold_sim.simulate(end_time, seed=42)
events = [torch.tensor(ev, dtype=torch.float) for ev in events]


print(list(map(len, events)))
print()
print(events[0])
print()
print(events[1])
print()
print(events[2])

[36977, 55857, 28262]

tensor([2.6265e+01, 2.9219e+01, 3.2827e+01,  ..., 9.9990e+04, 9.9992e+04,
        9.9994e+04])

tensor([1.6119e+01, 2.1493e+01, 2.5494e+01,  ..., 9.9995e+04, 9.9997e+04,
        9.9999e+04])

tensor([2.3371e+00, 8.2098e+00, 1.0009e+01,  ..., 9.9996e+04, 9.9997e+04,
        1.0000e+05])


In [6]:
sum(map(len, events))

121096

---

## Test variational inference algorithm

Create model object and set the data.

In [7]:
self = VariationalWoldModel(verbose=True)
self.set_data(events)

Define the parameters of the prior.

In [8]:
as_pr = 1.0 * np.ones((dim + 1, dim))
ar_pr = 1.0 * np.ones((dim + 1, dim))

bs_pr = 10.0 * np.ones((dim, dim))
br_pr = 10.0 * np.ones((dim, dim))

zc_pr = [1.0 * np.ones((len(events[i]), dim+1)) for i in range(dim)]

In [9]:
self.fit(as_pr=as_pr, ar_pr=ar_pr, bs_pr=bs_pr, br_pr=br_pr, zc_pr=zc_pr, max_iter=100, tol=1e-5)

-------------------------------------------------- 0
Alpha posterior mean:
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
Z[0] posterior probabilities
[[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 ...
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]
-------------------------------------------------- 1
Alpha posterior mean:
[[0.09 0.14 0.07]
 [0.09 0.14 0.07]
 [0.09 0.14 0.07]
 [0.09 0.14 0.07]]
Z posterior probabilities
[[0.54297 0.14663 0.18632 0.12407]
 [0.57175 0.1544  0.15058 0.12326]
 [0.75364 0.09164 0.07201 0.08271]
 ...
 [0.64634 0.03594 0.20965 0.10806]
 [0.58492 0.14796 0.1746  0.09253]
 [0.61762 0.11289 0.19927 0.07022]]
-------------------------------------------------- 2
Alpha posterior mean:
[[0.25 0.37 0.19]
 [0.14 0.21 0.11]
 [0.15 0.22 0.11]
 [0.14 0.2  0.11]]
Z posterior probabilities
[[0.74525 0.08024 0.10943 0.06508]
 [0.76161 0.082   0.0925  0.06388]
 [0.85268 0.05245 0.05003 0.04484]
 ...
 [0.80198 0.02849 0.1131  0.0

Alpha posterior mean:
[[0.37 0.56 0.28]
 [0.   0.01 0.  ]
 [0.01 0.01 0.  ]
 [0.   0.   0.  ]]
Z posterior probabilities
[[0.99762 0.00072 0.00119 0.00047]
 [0.99771 0.00072 0.00112 0.00046]
 [0.99828 0.00053 0.00088 0.00031]
 ...
 [0.99804 0.0004  0.00117 0.0004 ]
 [0.99777 0.0007  0.00115 0.00038]
 [0.9979  0.00062 0.00117 0.00032]]
-------------------------------------------------- 29
Alpha posterior mean:
[[0.37 0.56 0.28]
 [0.   0.01 0.  ]
 [0.   0.01 0.  ]
 [0.   0.   0.  ]]
Z posterior probabilities
[[0.99784 0.00064 0.00111 0.0004 ]
 [0.99793 0.00064 0.00104 0.00039]
 [0.9985  0.00046 0.0008  0.00024]
 ...
 [0.99825 0.00033 0.00109 0.00032]
 [0.99799 0.00063 0.00107 0.00031]
 [0.99812 0.00054 0.00109 0.00025]]
-------------------------------------------------- 30
Alpha posterior mean:
[[0.37 0.56 0.28]
 [0.   0.01 0.  ]
 [0.   0.01 0.  ]
 [0.   0.   0.  ]]
Z posterior probabilities
[[0.99806 0.00057 0.00103 0.00034]
 [0.99815 0.00057 0.00096 0.00032]
 [0.99871 0.00039 0.00073 0

Z posterior probabilities
[[0.99994 0.00002 0.00002 0.00002]
 [0.99995 0.00002 0.00001 0.00001]
 [1.      0.      0.      0.     ]
 ...
 [0.99998 0.      0.00002 0.00001]
 [0.99996 0.00002 0.00002 0.     ]
 [0.99997 0.00001 0.00002 0.     ]]
-------------------------------------------------- 59
Alpha posterior mean:
[[0.37 0.56 0.28]
 [0.   0.   0.  ]
 [0.   0.   0.  ]
 [0.   0.   0.  ]]
Z posterior probabilities
[[0.99994 0.00002 0.00002 0.00002]
 [0.99995 0.00002 0.00001 0.00001]
 [1.      0.      0.      0.     ]
 ...
 [0.99998 0.      0.00002 0.00001]
 [0.99996 0.00002 0.00002 0.     ]
 [0.99997 0.00001 0.00002 0.     ]]
-------------------------------------------------- 60
Alpha posterior mean:
[[0.37 0.56 0.28]
 [0.   0.   0.  ]
 [0.   0.   0.  ]
 [0.   0.   0.  ]]
Z posterior probabilities
[[0.99994 0.00002 0.00002 0.00002]
 [0.99995 0.00002 0.00001 0.00001]
 [1.      0.      0.      0.     ]
 ...
 [0.99998 0.      0.00002 0.00001]
 [0.99996 0.00002 0.00002 0.     ]
 [0.99997 0.

Alpha posterior mean:
[[0.37 0.56 0.28]
 [0.   0.   0.  ]
 [0.   0.   0.  ]
 [0.   0.   0.  ]]
Z posterior probabilities
[[0.99994 0.00002 0.00002 0.00002]
 [0.99995 0.00002 0.00001 0.00001]
 [1.      0.      0.      0.     ]
 ...
 [0.99998 0.      0.00002 0.00001]
 [0.99996 0.00002 0.00002 0.     ]
 [0.99997 0.00001 0.00002 0.     ]]
-------------------------------------------------- 82
Alpha posterior mean:
[[0.37 0.56 0.28]
 [0.   0.   0.  ]
 [0.   0.   0.  ]
 [0.   0.   0.  ]]
Z posterior probabilities
[[0.99994 0.00002 0.00002 0.00002]
 [0.99995 0.00002 0.00001 0.00001]
 [1.      0.      0.      0.     ]
 ...
 [0.99998 0.      0.00002 0.00001]
 [0.99996 0.00002 0.00002 0.     ]
 [0.99997 0.00001 0.00002 0.     ]]
-------------------------------------------------- 83
Alpha posterior mean:
[[0.37 0.56 0.28]
 [0.   0.   0.  ]
 [0.   0.   0.  ]
 [0.   0.   0.  ]]
Z posterior probabilities
[[0.99994 0.00002 0.00002 0.00002]
 [0.99995 0.00002 0.00001 0.00001]
 [1.      0.      0.      0

In [10]:
alpha_hat_mean = np.round(self._as_po / self._ar_po, 2)
baseline_hat = alpha_hat_mean[0,:]
adjacency_hat = alpha_hat_mean[1:,:]
beta_hat = np.round(self._br_po / (self._bs_po - 1), 2) - 1

print('Baseline:')
print('---------')
print('Ground truth:')
print(baseline)
print('Estimated:')
print(baseline_hat)
print()

print('Adjacency:')
print('---------')
print('Ground truth:')
print(adjacency)
print('Estimated:')
print(adjacency_hat)
print()

print('Beta:')
print('-----')
print('Ground truth:')
print(beta * np.ones((dim, dim)))
print('Estimated:')
print(beta_hat)

Baseline:
---------
Ground truth:
[0.1 0.1 0.2]
Estimated:
[0.37 0.56 0.28]

Adjacency:
---------
Ground truth:
[[0.3 0.8 0.1]
 [0.2 0.3 0.1]
 [0.2 0.1 0. ]]
Estimated:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

Beta:
-----
Ground truth:
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
Estimated:
[[0.25 0.25 0.25]
 [0.26 0.26 0.26]
 [0.25 0.25 0.25]]


In [11]:
self._as_po

array([[36977.2867 , 55857.24792, 28262.31186],
       [    1.22295,     1.24181,     1.21389],
       [    1.31934,     1.34121,     1.30943],
       [    1.17102,     1.16906,     1.16481]])

In [12]:
self._ar_po

array([[ 99994.73438, 100000.28125, 100000.92188],
       [ 88465.47251,  88326.98151,  88509.29679],
       [ 88083.3989 ,  88025.17852,  88119.79793],
       [ 88690.44754,  88719.54952,  88751.00208]])

In [13]:
self._bs_po

array([[10.22295, 10.24181, 10.21389],
       [10.31934, 10.34121, 10.30943],
       [10.17102, 10.16906, 10.16481]])

In [14]:
self._br_po

array([[11.55527, 11.59538, 11.54003],
       [11.71481, 11.74806, 11.69946],
       [11.46741, 11.46208, 11.4533 ]])

In [15]:
self._zp_po[0]

array([[0.99994, 0.00002, 0.00002, 0.00002],
       [0.99996, 0.00002, 0.00001, 0.00001],
       [0.99999, 0.     , 0.     , 0.     ],
       ...,
       [0.99998, 0.     , 0.00001, 0.00001],
       [0.99997, 0.00001, 0.00001, 0.     ],
       [0.99998, 0.     , 0.00001, 0.     ]])

---

In [16]:
from scipy.special import digamma

i = 0

as_po=self._as_po
ar_po=self._ar_po
bs_po=self._bs_po
br_po=self._br_po
delta_ikj=self.delta_ikj
last_t = self.last_t
zp_po=self._zp_po

In [17]:
zp_po[i].shape

(36213, 4)

In [18]:
br_po / (bs_po - 1)

array([[1.25277, 1.25426, 1.25243],
       [1.2569 , 1.2573 , 1.25626],
       [1.25046, 1.24999, 1.24958]])

In [19]:
bs_po

array([[10.22119, 10.23691, 10.21263],
       [10.31154, 10.3325 , 10.30481],
       [10.17248, 10.16832, 10.16512]])